# Querying

In [ ]:
!pip install -U pyTigerGraph

## Connect to Graph we want to query

In [ ]:
# Imports
import pyTigerGraph as tg

# fill in the below with your Tiger Graph credentials
HOSTNAME = "<ENTER_HOSTNAME>"
USERNAME = "<ENTER_USERNAME>"
PASSWORD = "<ENTER_PASSWORD>"
GRAPH_NAME = "taxonomy2"

# connect to pytigerGraph
conn = tg.TigerGraphConnection(
    host=HOSTNAME, username=USERNAME, password=PASSWORD, 
    gsqlVersion="3.0.5", 
    )

# set graph name to the graph we want to conneect to
conn.graphname = GRAPH_NAME

# create a secret
secret = conn.createSecret()

# use the secret to get a token
authToken = conn.getToken(secret)[0]

# connect to graph with token
conn = tg.TigerGraphConnection(
    host=HOSTNAME, username=USERNAME, password=PASSWORD, 
    graphname=GRAPH_NAME, apiToken=authToken
    )

### Test: Installing a query

In [ ]:
# create a query
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY sample_query() FOR GRAPH taxonomy2 {
    PRINT "Hello World";
}
""")

# install
conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY sample_query
""")

In [ ]:
# now run
result = conn.runInstalledQuery("sample_query")
print(result)

## Basic Queries
(with pyTigerGraph)

In [ ]:
# get vertices corresponding to an ID
results = conn.getVerticesById("Taxon", "10109647")
print(results) #[{'v_id': '10109647', 'v_type': 'Taxon', 'attributes': {'scientificName': 'Tobamovirus', 'canonicalName': '', 'genericName': '', 'specificEpithet': ''}}]


In [ ]:
# get edges of a taxon
results = conn.getEdgeCountFrom("Taxon", "10109647")
print(results) #{'parent_taxon_id': 1, 'child_taxon_id': 1, 'accepted_name_id': 1, 'original_name_id': 1, 'taxon_rank': 1, 'taxon_status': 1, 'taxon_distribution': 0, 'distribution_establishment': 0, 'distribution_occurrence': 0, 'distribution_locationid': 0, 'distribution_locality': 0, 'distribution_country': 0, 'sighting': 0}

In [ ]:
# get edges
results = conn.getEdges("Taxon", "10109647")
print(results) #[{'e_type': 'parent_taxon_id', 'directed': True, 'from_id': '10109647', 'from_type': 'Taxon', 'to_id': '6', 'to_type': 'Taxon', 'attributes': {}}, {'e_type': 'child_taxon_id', 'directed': True, 'from_id': '10109647', 'from_type': 'Taxon', 'to_id': '10574262', 'to_type': 'Taxon', 'attributes': {}}, {'e_type': 'accepted_name_id', 'directed': True, 'from_id': '10109647', 'from_type': 'Taxon', 'to_id': '-1', 'to_type': 'Taxon', 'attributes': {}}, {'e_type': 'original_name_id', 'directed': True, 'from_id': '10109647', 'from_type': 'Taxon', 'to_id': '-1', 'to_type': 'Taxon', 'attributes': {}}, {'e_type': 'taxon_rank', 'directed': False, 'from_id': '10109647', 'from_type': 'Taxon', 'to_id': 'genus', 'to_type': 'TaxonRank', 'attributes': {}}, {'e_type': 'taxon_status', 'directed': False, 'from_id': '10109647', 'from_type': 'Taxon', 'to_id': 'accepted', 'to_type': 'TaxonomicStatus', 'attributes': {}}]


## Generic Queries

Using GSQL interpreted queries. We can always convert the queries we use more often to 
installed queries for efficiency. 

TODO: Determine which queries are used more frequently, then create and install them for greater efficiency.

In [ ]:
# seleect all kingdom Taxon Ranks and return the connected Taxons
conn.runInterpretedQuery("""
INTERPRET QUERY get_all_kingdom_taxons() FOR GRAPH taxonomy2 {
    tr = {TaxonRank.*};
    result = SELECT t FROM tr - (taxon_rank) - Taxon:t
        WHERE tr.rank == "kingdom";
        
    PRINT result;
}
""")

In [ ]:
# 5 taxons whose taxon rank is species and taxon status is doubtful
# order of taxon ID
conn.runInterpretedQuery("""
INTERPRET QUERY get_all_doubtful_taxonstatus() FOR GRAPH taxonomy2 { 
    result = SELECT t FROM TaxonomicStatus:ts - (taxon_status) - Taxon:t - (taxon_rank) - TaxonRank:tr
        WHERE ts.status == "doubtful"
        AND t.genericName=="Acacia"
        AND tr.rank=="species"
        ORDER BY t.taxonID ASC
        LIMIT 5 OFFSET 1;
        
    PRINT result;
}
""")

In [ ]:
# count number of doubtful taxon statuses using accumulators
conn.runInterpretedQuery("""
INTERPRET QUERY count_doubtful_taxonstaus() FOR GRAPH taxonomy2 {
    
    SumAccum<INT> @@num_doubtful_taxonstatus;
    SetAccum<Vertex<TaxonomicStatus>> @@taxonomic_statuses;

    result = SELECT t FROM TaxonomicStatus:ts - (taxon_status) - Taxon:t
        WHERE ts.status == "doubtful"
        ACCUM
            @@num_doubtful_taxonstatus += 1,
            @@taxonomic_statuses += ts
        HAVING
            @@num_doubtful_taxonstatus > 1;
    
    PRINT @@num_doubtful_taxonstatus AS numDoubtfulTaxonStatus;
    PRINT @@taxonomic_statuses.size() AS numTaxonomicStatuses; # can omit the .size() to get the actual taxon itself
}
""")
# [{'numDoubtfulTaxonStatus': 273245}, {'numTaxonomicStatuses': 1}]

In [ ]:
# count number of unranked taxons
conn.runInterpretedQuery("""
INTERPRET QUERY count_unranked_taxons() FOR GRAPH taxonomy2 { 
  
    SumAccum<INT> @@num_unranked_taxons;

    result = SELECT t FROM TaxonRank:tr - (taxon_rank) - Taxon:t
        WHERE tr.rank == "unranked"
        ACCUM
            @@num_unranked_taxons += 1;

    
    PRINT @@num_unranked_taxons AS numUnrankedTaxon;
}
""")
# "numUnrankedTaxon": 778993


In [ ]:
# get 5 unranked taxons
conn.runInterpretedQuery("""
INTERPRET QUERY get_unranked_taxons() FOR GRAPH taxonomy2 { 
  /* Write query logic here */   
    result = SELECT t FROM TaxonRank:tr - (taxon_rank) - Taxon:t
        WHERE tr.rank == "unranked"
        LIMIT 5;
    
    PRINT result;
}
""")

### Taxon Queries

In [ ]:
# get specific taxon by id
conn.runInterpretedQuery("""
INTERPRET QUERY get_specific_taxon_by_id(UINT taxonID) FOR GRAPH taxonomy2 { 
  
    result = SELECT t FROM Taxon:t
        WHERE t.taxonID == taxonID
        LIMIT 1;
    
    PRINT result;
}
""", params={"taxonID": 10763537}) # example taxonid

In [ ]:
# find all the countries a taxonid is distributed in
# can be used to confirm if a taxa is present in a specific country
# example of multi-hop query
conn.runInterpretedQuery("""
    INTERPRET QUERY get_country_distributions_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 { 

  
    result = SELECT c FROM Taxon:t - (taxon_distribution) - Distribution - (distribution_country) - Country:c;
    
    PRINT result;
}
""", params={ "taxonID": 10763537 }) # example taxonid

In [ ]:
# get parent taxonid
# example of a directed traversal
conn.runInterpretedQuery("""
INTERPRET QUERY get_parent_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 { 
  
    result = SELECT pt FROM Taxon:t - (parent_taxon_id) -> Taxon:pt
        WHERE t.taxonID == taxonID
        LIMIT 1;
  
  PRINT result;
}
""", params={ "taxonID": 10763537 })

In [ ]:
# get all children of a taxonid
conn.runInterpretedQuery("""
INTERPRET QUERY get_children_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 { 
      
  SumAccum<INT> @@num_children_taxons;
  SetAccum<Vertex<Taxon>> @@children_taxon_ids;

  result = SELECT ct FROM Taxon: t - (child_taxon_id) -> Taxon: ct
    WHERE t.taxonID == taxonID
    ACCUM
        @@num_children_taxons+=1,
        @@children_taxon_ids += ct;
  
  PRINT @@num_children_taxons AS numChildrenTaxons;
  PRINT @@children_taxon_ids AS childrenTaxonIds;
}
""", params={ "taxonID": 3338 })

In [ ]:
# get sibling taxonids; returns result taxon, with siblings in @taxons
# Note; This query must be created
conn.gsql("""
CREATE QUERY get_taxonid_neighbours(UINT taxonID) FOR GRAPH taxonomy2 { 

  BagAccum<VERTEX> @child_taxon_ids;
  
  result = SELECT pt from Taxon:t - (parent_taxon_id) - Taxon:pt
    WHERE t.taxonID == taxonID 
    POST-ACCUM
      pt.@child_taxon_ids = pt.neighbors("child_taxon_id");
  
  PRINT result;
}
""")

In [ ]:
# install the above query
conn.gsql("""USE GRAPH taxonomy2 INSTALL QUERY get_taxonid_neighbours""")

In [ ]:
conn.runInstalledQuery("get_taxonid_neighbours", params={'taxonID': 10763537})

In [ ]:
# recursively get all parents of a taxon id as far as we can go
# example of a recursive traversal
conn.runInterpretedQuery("""
INTERPRET QUERY get_parents_recursive_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 SYNTAX v2{ 
  
    SumAccum<INT> @@num_taxon_parents;
    SetAccum<INT> @@taxon_parents_id;

    result = SELECT pt FROM Taxon:t - (parent_taxon_id>*) - Taxon:pt
        WHERE t.taxonID == taxonID
        ACCUM
            @@num_taxon_parents += 1,
            @@taxon_parents_id += pt.taxonID;

    PRINT @@num_taxon_parents AS numTaxonParents;
    PRINT @@taxon_parents_id as taxonParentsId;
}
""", params={ "taxonID": 10763537 })
# [{'numTaxonParents': 7}, {'taxonParentsId': [0, 1, 54, 216, 811, 3338, 10763537]}]

In [ ]:
# get ranking of taxonid
conn.runInterpretedQuery("""
INTERPRET QUERY get_taxonid_ranking(UINT taxonID) FOR GRAPH taxonomy2 { 
  result = SELECT tr FROM Taxon: t - (taxon_rank) - TaxonRank: tr
  WHERE t.taxonID==taxonID;
  
  PRINT result;
}
""", params={ "taxonID": 10763537 })
#[{'result': [{'attributes': {'rank': 'unranked'},'v_id': 'unranked','v_type': 'TaxonRank'}]}]

In [ ]:
# get taxon status of taxonid
conn.runInterpretedQuery("""
INTERPRET QUERY get_taxonid_status(UINT taxonID) FOR GRAPH taxonomy2 { 
  result = SELECT ts FROM Taxon: t - (taxon_status) - TaxonomicStatus: ts
  WHERE t.taxonID==taxonID;
  
  PRINT result;
}
""", params={ "taxonID": 10763537 })
#[{'result': [{'attributes': {'status': 'accepted'}, 'v_id': 'accepted', 'v_type': 'TaxonomicStatus'}]}]

Can get all the parents, then separate queries to get all their rankings, statuses etc..

### Player Sighting Queries

In [ ]:
# create new player (when a new player joins)
# can also be used to update player data
conn.runInterpretedQuery("""
INTERPRET QUERY create_player(STRING name, UINT level, UINT score) FOR GRAPH taxonomy2 { 
  /* the name will be the same as the ID for player. Perhaps in future this may be different. 
  Maybe the name can be the PK for Player as a better schema.*/ 
  INSERT INTO Player VALUES (name, name, level, score);
}
""", params={"name": "Nina", "level": 1, "score": 100})

In [ ]:
# get player info by id
conn.runInterpretedQuery("""
INTERPRET QUERY get_player_info(STRING playerID) FOR GRAPH taxonomy2 { 
  
  result = SELECT p from Player:p 
    WHERE p.id==playerID;
  PRINT result;
}
""", params={'playerID': "Hari"})

In [ ]:
# create a player sighting edge (connects player to taxon)
conn.runInterpretedQuery("""
INTERPRET QUERY create_sighting_edge(STRING playerID, UINT taxonID, FLOAT latitude, FLOAT longitude, DATETIME sightingTime, STRING img_store_url) FOR GRAPH taxonomy2 { 
  INSERT INTO sighting VALUES (playerID, taxonID, latitude, longitude, sightingTime, img_store_url);
}
""", params={
    'playerID': "Nina", 
    'taxonID': 10763537, 
    'latitude':'51.5', 
    'longitude':'0.13', 
    'sightingTime':'2022-01-01 00:00:00', 
    'img_store_url':'https://static.dw.com/image/47967537_101.jpg'
    })

In [ ]:
# get all sightings of a player
conn.runInterpretedQuery("""
INTERPRET QUERY get_all_player_sightings(STRING playerID) FOR GRAPH taxonomy2 { 
  
  SetAccum<EDGE> @@sighting_edges;

  result = SELECT p FROM Player:p - (sighting:s) - Taxon:t
    WHERE p.id == playerID
    ACCUM
      @@sighting_edges += s;
  
  PRINT @@sighting_edges;
}
""", params={'playerID': "Hari"})

In [ ]:
# create a guild; a guild has to be attached to a player also
# need to check that the player does not already belong to a guild first.
# player can only belong to one guild
conn.runInterpretedQuery("""
INTERPRET QUERY create_guild(STRING guildName, STRING playerId, DATETIME join_datetime) FOR GRAPH taxonomy2 { 
  /* Write query logic here */ 
  INSERT INTO Guild VALUES (guildName);
  INSERT INTO player_guild VALUES(playerId, guildName, join_datetime);
}
""", params={
        'guildName':'DevAdventurers2',
        'playerId':'Nina',
        'join_datetime':'2022-01-01 00:00:00'
})

In [ ]:
# get guild(s) of player
conn.runInterpretedQuery("""
INTERPRET QUERY get_player_guild(STRING playerID) FOR GRAPH taxonomy2 { 
  
  result = SELECT g FROM Player:p - (player_guild) - Guild:g
    WHERE p.id==playerID;
  
  PRINT result; 
}
""", params={'playerId':'Hari'})

In [ ]:
# get all guilds
conn.runInterpretedQuery("""
INTERPRET QUERY get_all_guilds() FOR GRAPH taxonomy2 { 
  result = SELECT g FROM Guild:g;
  PRINT result;
}
""")

In [ ]:
# connect other player to guild
# note; this can be also be used to create a guild if the guildname does not exist
# creates a new guild, so the previous query above is obsolete
# this also creates a new player if a different player id is entered that doesn't exist
conn.runInterpretedQuery("""
INTERPRET QUERY create_edge_player_guild(STRING playerID, STRING guildName, DATETIME join_datetime) FOR GRAPH taxonomy2 { 
  INSERT INTO player_guild VALUES(playerID, guildName, join_datetime);
}
""", params={
        'guildName':'DevAdventurers2',
        'playerId':'Wing',
        'join_datetime':'2022-01-01 00:00:00'
})

In [ ]:
# delete an edge
# dedlete player_guild edge (ie. when a player leaves a guild)
conn.runInterpretedQuery("""
INTERPRET QUERY delete_player_guild_edge(STRING playerID, STRING guildName) FOR GRAPH taxonomy2 { 
  
  result = SELECT g FROM Player:p - (player_guild:pg) - Guild:g
    WHERE g.name==guildName AND p.id==playerID
    ACCUM 
      DELETE (pg);
}
""", params={'playerId':'Wing', 'guildName':'DevAdventurers2'})

In [ ]:
# delete a guild
conn.runInterpretedQuery("""
INTERPRET QUERY delete_guild_node(STRING guildName) FOR GRAPH taxonomy2 { 
  result = SELECT g from Guild:g WHERE g.name==guildName POST-ACCUM DELETE(g);
}
""", params={'guildName':'DevAdventurers2'})

In [ ]:
# get all players in a guild
conn.runInterpretedQuery("""
INTERPRET QUERY get_players_in_guild(STRING guildName) FOR GRAPH taxonomy2 {
  result = SELECT p from Guild:g - (player_guild) - Player:p
    WHERE g.name==guildName;
  PRINT result; 
}
""", params={'guildName':'DevAdventurers'})

In [ ]:
# see all the unique taxons a guild has collected
conn.runInterpretedQuery("""
INTERPRET QUERY get_guild_all_sightings(STRING guildName) FOR GRAPH taxonomy2 { 
    
  result = SELECT t FROM Guild:g - (player_guild) - Player:p - (sighting:s) - Taxon:t
    WHERE g.name==guildName;
  
  PRINT result;
}
""", params={'guildName':'DevAdventurers'})

### Notes:
-  For the app; of 10,000 original species, how many have you spotted?
-  GBIF Species description API: https://api.gbif.org/v1/species/5231190/descriptions
-  GBIF Species API: https://api.gbif.org/v1/species/5231190

## Create and Install Queries for FrontEnd
With the above we have shown what interpreted queries can do; now we will install the ones we wish to use directly for v1 of our frontend app.

In [ ]:
# install create sighting edge
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY create_sighting_edge(STRING playerID, UINT taxonID, FLOAT latitude, FLOAT longitude, DATETIME sightingTime, STRING img_store_url) FOR GRAPH taxonomy2 { 
  INSERT INTO sighting VALUES (playerID, taxonID, latitude, longitude, sightingTime, img_store_url);
}
""")

# install
conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY create_sighting_edge
""")

In [ ]:
# install get all guilds
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY get_all_guilds() FOR GRAPH taxonomy2 { 
  result = SELECT g FROM Guild:g;
  PRINT result;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY get_all_guilds
""")

In [ ]:
# install get all player sightings
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY get_all_player_sightings(STRING playerID) FOR GRAPH taxonomy2 { 
  
  SetAccum<EDGE> @@sighting_edges;

  result = SELECT p FROM Player:p - (sighting:s) - Taxon:t
    WHERE p.id == playerID
    ACCUM
      @@sighting_edges += s;
  
  PRINT @@sighting_edges;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY get_all_player_sightings
""")

In [ ]:
# install get children of taxonid
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY get_children_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 { 
      
  SumAccum<INT> @@num_children_taxons;
  SetAccum<Vertex<Taxon>> @@children_taxon_ids;

  result = SELECT ct FROM Taxon: t - (child_taxon_id) -> Taxon: ct
    WHERE t.taxonID == taxonID
    ACCUM
        @@num_children_taxons+=1,
        @@children_taxon_ids += ct;
  
  PRINT @@num_children_taxons AS numChildrenTaxons;
  PRINT @@children_taxon_ids AS childrenTaxonIds;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY get_children_of_taxonid
""")

In [ ]:
# install get country distributions for taxonid
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY get_country_distributions_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 { 

  
    result = SELECT c FROM Taxon:t - (taxon_distribution) - Distribution - (distribution_country) - Country:c;
    
    PRINT result;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY get_country_distributions_of_taxonid
""")

In [ ]:
# install get parent of taxonid
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY get_parent_of_taxonid(UINT taxonID) FOR GRAPH taxonomy2 { 
  
    result = SELECT pt FROM Taxon:t - (parent_taxon_id) -> Taxon:pt
        WHERE t.taxonID == taxonID
        LIMIT 1;
  
  PRINT result;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY get_parent_of_taxonid
""")

In [ ]:
# install get taxonid neighbours
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY get_taxonid_neighbours(UINT taxonID) FOR GRAPH taxonomy2 { 

  BagAccum<VERTEX> @child_taxon_ids;
  
  result = SELECT pt from Taxon:t - (parent_taxon_id) - Taxon:pt
    WHERE t.taxonID == taxonID 
    POST-ACCUM
      pt.@child_taxon_ids = pt.neighbors("child_taxon_id");
  
  PRINT result;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY get_taxonid_neighbours
""")

In [ ]:
# install new get sightings
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY new_get_sightings(STRING playerID) FOR GRAPH taxonomy2 SYNTAX v2 { 
    SELECT t.canonicalName, t, t.genericName, s INTO T
        FROM Player:p - (sighting:s) - Taxon:t
        WHERE p.id == playerID;
    PRINT T;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY new_get_sightings
""")

In [ ]:
# install new get taxon info
conn.gsql("""
USE GRAPH taxonomy2
CREATE QUERY new_get_taxon_info(UINT taxonID) FOR GRAPH taxonomy2 SYNTAX v2 { 
   SELECT s as status, r as rank, t.canonicalName, t, t.genericName INTO S
        FROM  TaxonRank:r - (taxon_rank) - Taxon:t - (taxon_status) - TaxonomicStatus:s
        WHERE t.taxonID == taxonID;


    PRINT S;
}
""")

conn.gsql("""
USE GRAPH taxonomy2
INSTALL QUERY new_get_taxon_info
""")